# Rote Learning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Der α-β-Pruning Algorithmus versucht durch die Auswertung der zukünftig möglichen Züge den Besten auszuwählen. Da der Suchraum jedoch zu groß ist um jedes mögliche Spiel zu betrachten, muss an einem bestimmten Punkt das Vorausschauen beendet werden und eine Schätzung statt dem tatsächlichen Wert verwendet werden. Diese Aufgabe übernimmt die in dem Notebook `nmm-heuristic` implementierte Heuristik.
Dies bedeutet dass eine künstliche Intelligenz, die den α-β-Pruning Algorithmus implementiert, je besser ist, desto mehr Schritte in die die Zukunft geschaut werden kann. Die Laufzeit α-β-Pruning Algorithmus nimmt jedoch mit zunehmender Tiefe, aufgrund der sehr schnell ansteigenden Anzahl der Zustände, deutlich zu. Somit ist eine Neuberechnung der Schäzung bei jedem Zug nicht praktikabel.

An diesem Punkt setzt das Rote-Learning an: Indem der α-β-Pruning Algorithmus um eine elementare Form des Lernens erweitert wird, wird dessen Effektivität stark gesteigert. Grundlegend werden bei der Verwendung von Rote-Learning bestimmte besuchte Zustände zusammen mit den dazugehörigen errechneten Schätzungen abgespeichert. Anstatt die Wertschätzung dieser Zustände bei jedem Zug neu zu berechnen, können diese nun aus dem Speicher abgerufen werden. Dies spart vor Allem bei einer hohen Suchtiefe viel Rechenzeit ein. Diese Einsparung der Rechenzeit kann darauf verwendet werden, weitere Zustände zu berechnen und somit die Qualität der Schätzungen zu erhöhen. Da mit jedem Spiel mehr Zustände und deren Schätzungen gespeichert werden, verbessert sich das Ergebnis des Algorithmus über Zeit. Es tritt also ein Lerneffekt ein.

Es gibt zwei Möglichkeiten Rote-Lerning zu implementieren: Einersteits kann eine primitive Version verwendet werden, die einfacherweise jeden besuchten Zustand zusammen mit der Rekursionstiefe und dessen Wert abspeichert. Andererseits kann der errechnete Wert am Grundzustand abgespeichert werden, dieser wird wenn möglich später anstatt der Heuristik weiterverwendet. Dadurch erhöht sich die effektive Suchtiefe. 

## Implementierung

Um das oben genannte Prinzip von Rote-Learning in Python zu implementieren, wurde auf eine bereits fertige Implementierung des α-β-Prunings Algorithmus für das Spiel Mühle zurück gegriffen. Dieser wurde im Rahmen der Studienarbeit entwickelt und musste für die Verwendung des primitiven Rote-Learnings nur noch geringfügig angepasst werden. Für das richtige Rote-Learning wurde α-β-Pruning neu implementiert. Um zu evaluieren ob Rote-Learning einen Vorteil gegenüber einem nicht trainierten Algorithmus herbeiführt, werden folgende Notebooks benötigt:

- `nmm-cache` implementiert eine schnelle, persistierbare Transpositionstabelle (auch Cache genannt);
- `nmm-cache-rote-learning` implementiert ebenfalls eine schnelle, persistierbare Transpositionstabelle, diese ist jedoch angepasst an das *Rote-Learning* (auch CacheRoteLearning genannt);
- `nmm-rote-training` implementierung ein Klasse für den Trainingsteil des Rote-Learnings;
- `nmm-alpha-beta-pruning` ist eine bereits vorhandene Implementierung des α-β-Pruning Algorithmus, welche ebenfalls Iterative Deepening und das primitive Rote-Learning verwendet;
- `nmm-alpha-beta-pruning-rote-learning` ist eine Implementierung des α-β-Pruning Algorithmus ohne Iterative Deepening, aber mit einer richtigen Rote-Learning Implementierung;
- `nmm-tournament` ist eine ebenfalls bereits vorhandene Implementierung zur Evaluation, ob ein per Rote-Learning trainierter Algorithmus besser abschneidet als ein nicht trainierter Algorithmus.

In [ ]:
%run ./nmm-cache.ipynb
%run ./nmm-cache-rote-learning.ipynb
%run ./nmm-rote-training.ipynb
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-alpha-beta-pruning-rote-learning.ipynb
%run ./nmm-tournament.ipynb

Desweitern wird das Paket `memory_profiler` eingebunden, welches die Überwachung der Auslastung des Arbeitsspeichers ermöglicht.

In [ ]:
from memory_profiler import memory_usage

## Primtives Rote-Learning

Primitives Rote-Learning (*This can hardly be called a very advanced form of learning* [Samuel1959, S. 215]) ist eine einfache Methode des Auswendiglernens, indem sich jeder besuchte Zustand samt Rekursionslimit und Wert abgespeichert wird. Zu jedem Zeitpunkt wird überprüft, ob der Cache den besuchten Zustand beinhaltet. Wenn ja, dann zählt der Zustand nicht zum Limit der zu besuchenden Zustände. Dies ermöglicht in Verbindung mit Iterative Deepening eine Verbesserung der Spielqualität, da effektiv mehr Zustände besucht werden können.

### Cache

Um den bei Rote-Learning beschriebenen Lerneffekt zu erzielen und die Zustände zu Speichern, wurde eine Transpositionstabelle als Klasse `Cache` in dem Notebook `nmm-cache` implementiert. Diese Klasse verfügt über fünf Methoden zur Verwaltung des Caches:

- `write` speichert einen neuen Zustand und die errechneten Werte ab;
- `read` liest die gespeicherten Werte eines Zustandes aus dem Cache aus;
- `save` legt den Inhalt des Caches in einer Datei auf einem Datenträger ab;
- `load` lädt die Daten des Caches aus einer Datei auf dem Datenträger;
- `clean` löscht Einträge aus dem Cache, falls die Anzahl der Einträge `max_size` überschritten wurde.

Um die einzelnen Werte für die Zustände im Cache abzulegen, werden sowohl die Zustände als auch die Werte in ein Byte-Array konvertiert. Insgesamt ist ein Eintrag des Caches somit 32 Bytes groß. Um zu verhindern, dass der Cache zu groß wird um ihn im Arbeitsspeicher oder auf dem Datenträger speichern zu können, wurde der Parameter `max_size` eingeführt. Dieser begrenzt den Cache auf eine maximale Anzahl an Einträgen. Um die Performance zu steigern, wird dies jedoch nicht beim jedem Aufruf der `write` Methode geprüft sondern nur beim Aufruf der Methode `clean`. Dies kann dazu führen, dass zwischen den Aufrufen der `clean` Methode mehr Einträge im Cache vorhanden sind, als durch `max_size` erlaubt.
Wird der Cache auf dem Datenträger abgelegt, entspricht die Dateigröße genau der Anzahl an Elementen multipliziert mit 32 Bytes. Die Größe das Caches im Arbeitsspeicher ist jedoch um einen Faktoren von ca. $6$ größer, da Python weiteren Arbeitsspeicher, beispielsweise für den Datentyp, reserviert.

Der folgende Befehl erstellt einen Cache mit einer Größe von 50 Millionen Zuständen.

In [ ]:
cache = Cache(max_size = 50_000_000)

### Anbindung an α-β-Pruning

Da im α-β-Pruning Algorithmus bereits die Verwendung eines In-Memory-Caches vorgesehen ist, mussten nur kleine Anpassungen vorgenommen werden. Es musste sichergestellt werden, dass sowohl zum Schreiben als auch zum Lesen der Werte aus dem Cache die richtigen Methoden aufgerufen werden, sowie dass ein Cache entweder als Parameter übergeben werden kann oder ein neuer instanziiert wird. Nach jeder Berechnung der nächsten besten Züge mit Hilfe der `bestMoves` Methode, wurde der Aufruf der `clean` Methode hinterlegt, um die Größe des Caches anzupassen.

###  Traingingsprozess

Um die Transpositionstabelle zu trainieren und somit das Rote-Learning umzusetzen, wird die Klasse `Training` aus dem Notebook `nmm-rote-training` verwendet. Dazu muss zuerst eine Methode erstellt werden, welche den Algorithmus für die künstliche Intelligenz konfiguriert und generiert. Diese Methode wird `artificial_intelligence_generator` genannt und generiert eine AlphaBetaPruning Instanz, welche den übergebenen Cache und eine benutzerdefinierte Heuristiken verwendet. Die verwendeten Heuristiken wurden bereits im Rahmen der Studienarbeit ermittelt.

In [ ]:
def artificial_intelligence_generator_primitiv(cache: Cache):
    customWeights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1)
    return AlphaBetaPruning(cache = cache, weights = customWeights)

Für den Cache wurde sich für eine maximale Größe von 50.000.000 Einträgen entschieden. Dies resultiert in eine maximal Größe des Caches von ca. 1,6Gb auf dem Datenträger. Des Weiteren werden die Standardwerte der Training-Klasse verwendet. Das heißt, dass ingesamt 100 Spiele gespielt werden und der Seed für den Zufallsgenerator nicht angepasst wird. Alle 10 Spiele wird der Cache auf dem Datenträger gespeichert und wird der Prefix `training-primitive-` für den Dateinamen verwendet.

In [ ]:
training = Training(
    cache = cache,
    trainings = 100,
    max_turns = 18,
    artificial_intelligence = artificial_intelligence_generator_primitiv,
    path_prefix = 'training-primitive-'
)

Der Trainingsprozess wird durch den Aufruf der Funktion `train` gestartet und dauert mit der oben genannten Konfiguration in etwa 4,5 Stunden. Dabei werden bis zu 10 Gb Arbeitsspeicher benötigt.

In [ ]:
mem_usage = memory_usage(training.train)
print('Maximum memory usage: %s MB.' % max(mem_usage))

### Trainings Limitierungen

Auf den ersten Blick scheint es so, dass man den Cache unendlich weiter trainieren kann. Dies jedoch mit der aktuellen Implementierung nicht zielführend. Ab einem gewissen Trainingszeitpunkt ist das Rekursionslimit der Einträge im Cache so hoch, dass dieses durch weiteres Training nicht erneut erreicht werden können. Dies ist lässt sich gut an nachfolgendem Beispiel erkennen:

Durch die geringe Größe des Caches sowie dem kleinen Wert für `max_states` kommt man sehr schnell an den Punkt, an dem das Cache-Limit auf 3 erhöht wird. Dadurch werden fast alle Einträge aus dem Cache gelöscht und es verbleiben nur noch `48` im Cache. Dies passiert mehrere Male und ist ein Zeichen dafür, dass der Cache nicht mehr weiter trainiert werden kann, da dieser zu klein ist um genügend Einträge zu halten, damit mehr Einträge mit `limit = 3` berechnet werden können bevor der `max_states` Wert überschritten wird.

In [ ]:
training = Training(
    cache                   = Cache(max_size = 1_000),
    artificial_intelligence = lambda cache: AlphaBetaPruning(
        cache      = cache,
        max_states = 1_000
    ),
    max_turns               = 100,
    path_prefix             = "max-learning-",
    save_interval           = 1
)
training.train()

## Rote-Learning

### CacheRoteLearning

Der CacheRoteLearning funktioniert äquivalent zum normalen Cache, allerdings wird ledeglich der Zustand und der errechnete Wert abgespeichert: Weder das Rekurstionslimit noch Alpha und Beta werden gespeichert. Da dieser Cache sehr langsam wächst, war eine implementierung der `clean` Funktion nicht nötig. Dadurch ergeben sich folgende Funktionen:

- `write` speichert einen neuen Zustand und die errechneten Werte ab;
- `read` liest die gespeicherten Werte eines Zustandes aus dem Cache aus;
- `save` legt den Inhalt des Caches in einer Datei auf einem Datenträger ab;
- `load` lädt die Daten des Caches aus einer Datei auf dem Datenträger.

In [ ]:
cacheRoteLearning = CacheRoteLearning()

### α-β-Pruning

Da die ursprüngliche Implementierung des α-β-Pruning Algorithmus Iterative Deepening verwendet, welches nur funktioniert wenn jeder Zustand abgespeichert wird, musste α-β-Pruning neu implementiert werden ohne dass Iterative Deepening verwendet wird. Das normale Rote-Learning speichert den errechneten Wert eines Zustandes ab, sobald alle Zustände betrachtet wurden, also am Ende der `bestMoves` funktion. Sobald das Rekursionslimit bei der Berechnung erreicht wurde, wird entweder der Eintrag aus dem Cache gelesen oder die Heuristik verwendet, wenn kein Eintrag verfügbar ist.

### Traininsprozess

Auch hier wird wieder die `Training` Klasse verwendet um das normale Rote-Learning zu trainieren. Dazu wird eine Funktion erstellt, die mit einem gegebenem `CacheRoteLearning` eine neue Instanz der `AlphaBetaPruningRoteLearning` Klasse mit den korrekten Einstellungen erstellt.

In [ ]:
def artificial_intelligence_generator_rote_learning(cache: CacheRoteLearning):
    customWeights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1)
    return AlphaBetaPruningRoteLearning(cache = cacheRoteLearning, weights = customWeights)

Im Gegensatz zum primiten Rote-Learning soll hier der komplette Spielverlauf trainiert werden, damit festgestellt werden kann, ob diese Form des Trainings eine Auswirkung hat. Diese Entscheidung wurde getroffen, da im Vergleich zum primitiven Rote-Learning viel weniger Zustände im Cache abgelegt werden und sich somit eine bessere Vergleichbarkeit der trainierten und untrainierten Instanzen erhofft wurde.

In [ ]:
training = Training(
    cache = cacheRoteLearning,
    trainings = 100,
    max_turns = 250,
    artificial_intelligence = artificial_intelligence_generator_rote_learning,
    path_prefix = 'training-rote-learning-'
)

Um das Training zu starten wird die Funktion `train` aufgerufen. Dieser Prozess dauert ca. 2 Stunden.

In [ ]:
mem_usage = memory_usage(training.train)
print('Maximum memory usage: %s MB.' % max(mem_usage))

## Auswertung

Bei der Auswertung wird auf die Klasse `Tournament` aus dem Notebook `nmm-tournament` zurück gegriffen. Diese Klasse besitzt die Methode `play`, welche es ermöglicht verschiedene Algorithmen gegeneinander Antreten zu lassen und die Ergebnisse zu speichern.

Um zu ermitteln ob die verschiedenen Rote-Learning-Methoden einen Vorteil bieten, treteb verschiedene trainierte und untrainierte Transpositionstabellen gegeneinander an. Dies wird zehn mal mit verschiedenen Seeds wiederholt. Das verändern der Seeds sorgt dabei für einen veränderten Spielverlauf. Bei jedem Spiel spielt jeder Algorithmus einmal als weißer und einmal als schwarzer Spieler. Somit sollte ermittelt werden können, ob durch das Training eine Verbesserung der künstlichen Intelligenz eintritt und wenn ja, wie deutlich diese Verbesserung ausfällt.

### Trainiertes primtives Rote-Learning vs. Untrainiertes primtives Rote-Learning

In [ ]:
for i in range(10):
    Tournament(
        [
            lambda: AlphaBetaPruning(
                name='Primitive Rote-Learning Untrained',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000)
            ),
            lambda: AlphaBetaPruning(
                name='Primitive Rote-Learning Trained',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000, path = 'training-primitive-final.cache')
            )
        ],
        instances_per_round = 1,
        seed_offset         = i,
        name                = f"primtive-{i}-seed"
    ).play()

Die Auswertung hat ergeben, dass der trainierte Algorithmus von insgesamt 20 gespielten Spielen 10 gewonnen wurden. Bei 5 lief es auf ein Unentschieden herraus und 5 wurden verloren. Damit lässt sich mit ziemlicher Sicherheit sagen, dass das Training der ersten Spielphase durch Rote-Learning einen Vorteil im gesamten Spielverlauf bietet.

### Trainiertes Rote-Learning vs. Untrainiertes Rote-Learning

In [ ]:
for i in range(10):
    Tournament(
        [
            lambda: AlphaBetaPruningRoteLearning(
                name='Rote-Learning Untrained',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = CacheRoteLearning()
            ),
            lambda: AlphaBetaPruningRoteLearning(
                name='Rote-Learning Trained',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = CacheRoteLearning(path = 'training-rote-learning-final.cache')
            )
        ],
        instances_per_round = 1,
        seed_offset         = i,
        name                = f"rote-learning-{i}-seed"
    ).play()

Der trainierte Algorithmus hat 7 Spiele gewonnen und 4 Spiele verloren. Die restlichen Spiele endeten in einem Remis. Damit ist die Wahrscheinlichkeit hoch, dass der das Rote-Learning einen Vorteil im Spiel bietet.

## Fazit
Das Ziel dieser Arbeit war die verschiedenen Implementierungen des in dem Paper `Some Studies in Machine Learning Using the Game of Checkers` von `A.L.Samuel` beschriebene Prinzip von Rote-Learning zu erarbeiten. Das Ziel wurde erfolgreich durch eine Python-Implementierung erreicht und durch eine Verbesserung der Leistung einer künstlichen Intelligenz bestätigt. Im Laufe der Arbeit wurde sich neben der Implementierung in Python auch mit den dem α-β-Pruning Algorithmus sowie verschiedenen Methoden zur Persistierung eines Caches beschäftigt.

Während in dem Paper jedoch zwei Methoden beschrieben werden, um Einträge aus einem zu großen Cache zu löschen, wird sich in dieser Arbeit bei dem primitiven Rote-Learning nur auf eine Methode konzentriert. Im Paper wird zum einen die Aufruf-Frequenz oder zum anderen das Rekursionslimit des Eintrags. Diese Arbeit behandelt nur das Löschen von Einträgen basierend auf dem Rekursionslimit. Für die Implementierung des richtigen Rote-Learnings war keine Löschung der Einträge nötig, da die Cache-Größe nicht stark gewachsen ist. Sollte eine Löschung dennoch nötig werden, dann ist die Verwendung der Aufruf-Frequenz nötig, da das Rekursionslimit auf einen festen Wert gesetzt wurde.

Dennoch lässt sich sagen, dass die Rote-Learning Algorithmen erfolgreich waren, da eine Verbesserung des α-β-Pruning mit einem trainierten Cache erreicht werden konnte. In beiden Fällen könnte die trainierte künstliche Intelligenz öfters gewinnen als die untrainierte künstliche Intelligenz.